# Data source

## Objective

The primary objective of this notebook is to process the raw Ames Housing dataset and prepare it for downstream modeling. We will begin by ingesting the data, conducting initial data validation to assess its structure and quality, and then addressing issues like missing values and feature inconsistencies.

## Time

You should expect to spend (TODO: add expected runtime) running this sample.

### Dataset

[Kaggle Repo](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

This dataset has 79 explanatory variables that describe many aspects of residential homes in Ames, Iowa. Our target variable is `SalePrice`.

## Before you begin

## Installation

Install the packages required to execute this notebook.

In [2]:
! pip3 install --quiet --upgrade pip
! pip3 install --quiet -r ../requirements.txt

## Imports/Configuration

In [3]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# only display up to 3 decimal points when pd is used in print()
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))

in_file_paths = {
    'train': '../data/house-prices-advanced-regression-techniques/train.csv',
    'test': '../data/house-prices-advanced-regression-techniques/test.csv',
    'descriptions': '../data/house-prices-advanced-regression-techniques/data_description.txt',
}

out_file_paths = {
    'root': './outputs',
    'train': './outputs/1_train.csv',
    'test': './outputs/1_test.csv',
}

# create output directory if it does not exist already
os.makedirs(out_file_paths['root'], exist_ok=True)

## Step 1: Data Ingestion

In [4]:
train = pd.read_csv(in_file_paths['train'], index_col='Id')
test = pd.read_csv(in_file_paths['test'], index_col='Id')
print(f"successfully read dataset...\ntrain shape = {train.shape}\ntest shape = {test.shape}")
train.head()

successfully read dataset...
train shape = (1460, 80)
test shape = (1459, 79)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
train.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000,1201.000,1460.000,1460.000,1460.000,1460.000,1460.000,1452.000,1460.000,1460.000,...,1460.000,1460.000,1460.000,1460.000,1460.000,1460.000,1460.000,1460.000,1460.000,1460.000
mean,56.897,70.050,10516.828,6.099,5.575,1971.268,1984.866,103.685,443.640,46.549,...,94.245,46.660,21.954,3.410,15.061,2.759,43.489,6.322,2007.816,180921.196
std,42.301,24.285,9981.265,1.383,1.113,30.203,20.645,181.066,456.098,161.319,...,125.339,66.256,61.119,29.317,55.757,40.177,496.123,2.704,1.328,79442.503
min,20.000,21.000,1300.000,1.000,1.000,1872.000,1950.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,2006.000,34900.000
25%,20.000,59.000,7553.500,5.000,5.000,1954.000,1967.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.000,2007.000,129975.000
50%,50.000,69.000,9478.500,6.000,5.000,1973.000,1994.000,0.000,383.500,0.000,...,0.000,25.000,0.000,0.000,0.000,0.000,0.000,6.000,2008.000,163000.000
75%,70.000,80.000,11601.500,7.000,6.000,2000.000,2004.000,166.000,712.250,0.000,...,168.000,68.000,0.000,0.000,0.000,0.000,0.000,8.000,2009.000,214000.000
max,190.000,313.000,215245.000,10.000,9.000,2010.000,2010.000,1600.000,5644.000,1474.000,...,857.000,547.000,552.000,508.000,480.000,738.000,15500.000,12.000,2010.000,755000.000


## Cleaning

We will now deal with missing values.

In [6]:
nanDict = {
    'Total': train.isnull().sum(),
    'Pctg': train.isnull().sum() / train.isnull().count(),
    'Type': train.dtypes
}

nanStats = pd.DataFrame.from_dict(nanDict).sort_values(by=['Total', 'Pctg'], ascending=False)
nanStats = nanStats[nanStats['Total'] > 0]
nanStats

,Total,Pctg,Type
PoolQC,1453,0.995,object
MiscFeature,1406,0.963,object
Alley,1369,0.938,object
Fence,1179,0.808,object
MasVnrType,872,0.597,object
FireplaceQu,690,0.473,object
LotFrontage,259,0.177,float64
GarageType,81,0.055,object
GarageYrBlt,81,0.055,float64
GarageFinish,81,0.055,object


In [7]:
# make a copy of original train data
baseTrain = train.copy()

# We should be OK to drop PoolQC, since it is ~99% NANs
train.drop('PoolQC', axis=1, inplace=True, errors='ignore')

In [8]:
categoricalCols = train.select_dtypes(exclude='number').columns

# fill categorical NANs with 'None'
train[categoricalCols] = train[categoricalCols].fillna('None')
train[categoricalCols].head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,Fence,MiscFeature,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,None,Attchd,RFn,TA,TA,Y,None,None,WD,Normal
2,RL,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,TA,Attchd,RFn,TA,TA,Y,None,None,WD,Normal
3,RL,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,TA,Attchd,RFn,TA,TA,Y,None,None,WD,Normal
4,RL,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Gd,Detchd,Unf,TA,TA,Y,None,None,WD,Abnorml
5,RL,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,TA,Attchd,RFn,TA,TA,Y,None,None,WD,Normal


In [9]:
numericalCols = train.select_dtypes(include='number').columns
nanNumericColumns = [col for col in numericalCols if train[col].isnull().any()]
train[train[nanNumericColumns].isnull().any(axis=1)][nanNumericColumns]

,LotFrontage,MasVnrArea,GarageYrBlt
Id,,,
8,NaN,240.000,1973.000
13,NaN,0.000,1962.000
15,NaN,212.000,1960.000
17,NaN,180.000,1970.000
25,NaN,0.000,1968.000
...,...,...,...
1444,NaN,0.000,1916.000
1447,NaN,189.000,1962.000
1450,21.000,0.000,NaN


We will assume `LotFrontage` (linear feet of street connected to property) is `NaN` when there is no street connected - so we can fill these `NaN`'s with 0's.

In [10]:
train.LotFrontage.fillna(0, inplace=True)
train.LotFrontage.sort_values().unique()[:5]

array([ 0., 21., 24., 30., 32.])

In [11]:
train[train.MasVnrArea.isna()][['MasVnrArea', 'MasVnrType']]

,MasVnrArea,MasVnrType
Id,,
235,NaN,None
530,NaN,None
651,NaN,None
937,NaN,None
974,NaN,None
978,NaN,None
1244,NaN,None
1279,NaN,None


Whenever `MasVnrArea` is `NaN`, it has a corresponding `MasVnrType` of `None`. We will fill these `NaN`'s with 0's.

In [12]:
train.MasVnrArea.fillna(0, inplace=True)
train.MasVnrArea.sort_values().unique()[:5]

array([ 0.,  1., 11., 14., 16.])

Finally we deal with `GarageYrBlt`. This is always `NaN` when `GarageType` == `None`, so it shouldn't technically matter what we fill these with. Since we decided to eliminate `NaN`'s we will just fill with 0.

In [13]:
# train[train.GarageType == 'None'][['GarageType', 'GarageYrBlt']]
train[train.GarageYrBlt.isna()][['GarageType', 'GarageYrBlt']]

,GarageType,GarageYrBlt
Id,,
40,None,NaN
49,None,NaN
79,None,NaN
89,None,NaN
90,None,NaN
...,...,...
1350,None,NaN
1408,None,NaN
1450,None,NaN


In [14]:
train.GarageYrBlt.fillna(0, inplace=True)
train.GarageYrBlt.sort_values().unique()[:5]

array([   0., 1900., 1906., 1908., 1910.])

In [15]:
train.columns[train.isna().any()]

Index([], dtype='object')

## Dropping irrelevant features

We can estimate what features will be relevant/irrelevant for predicting `SalePrice` by using [`mutual_info_regression`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html).

In [16]:
def get_mutual_info(X, y):
    from sklearn.feature_selection import mutual_info_regression
    X = X.copy()

    # bool mask for if a column is categorical
    catTypes = ["object", "category"]
    catMask = X.apply(lambda x: x.dtype in catTypes)

    # encode categoricals as numerical values
    catCols = X.select_dtypes(catTypes).columns
    X[catCols] = X[catCols].apply(lambda col: col.factorize()[0])

    # compute mutual info
    miScores = mutual_info_regression(X, y, discrete_features=catMask, random_state=0)
    miScores = pd.Series(miScores, name="MI Scores", index=X.columns).sort_values(ascending=False)
    return miScores

In [17]:
X = train.copy()
y = X.pop('SalePrice')
mi = get_mutual_info(X, y)
mi.tail(10)

MiscVal        0.005
LowQualFinSF   0.004
LandSlope      0.003
Street         0.001
Condition2     0.000
Utilities      0.000
BsmtFinSF2     0.000
PoolArea       0.000
MoSold         0.000
YrSold         0.000
Name: MI Scores, dtype: float64

In [18]:
# drop columns with 0 MI score
columns = mi[mi == 0].index
X.drop(columns, axis=1, inplace=True, errors='ignore')
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,Fence,MiscFeature,MiscVal,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.000,8450,Pave,None,Reg,Lvl,AllPub,Inside,...,0,61,0,0,0,None,None,0,WD,Normal
2,20,RL,80.000,9600,Pave,None,Reg,Lvl,AllPub,FR2,...,298,0,0,0,0,None,None,0,WD,Normal
3,60,RL,68.000,11250,Pave,None,IR1,Lvl,AllPub,Inside,...,0,42,0,0,0,None,None,0,WD,Normal
4,70,RL,60.000,9550,Pave,None,IR1,Lvl,AllPub,Corner,...,0,35,272,0,0,None,None,0,WD,Abnorml
5,60,RL,84.000,14260,Pave,None,IR1,Lvl,AllPub,FR2,...,192,84,0,0,0,None,None,0,WD,Normal


## Summary

The transformations we made on the original dataset were:

1. Filling categorical `NaN`'s with `'None'`
2. Filling numerical `NaN`'s with 0
3. Dropped columns independent of `SalePrice` (via [`mutual_info_regression`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html) along with `PoolQC`), i.e., `[Utilities, BsmtFinSF2, PoolArea, MoSold, YrSold, PoolQC]`

We will end this notebook by 

In [19]:
def fill_nans(df):
    df = df.copy()
    # fill categorical NANs with 'None'
    categoricalCols = df.select_dtypes(exclude='number').columns
    df[categoricalCols] = df[categoricalCols].fillna('NONE')
    # fill numeric NANs with 0
    numericalCols = df.select_dtypes(include='number').columns
    df[numericalCols] = df[numericalCols].fillna(0)
    
    return df

def drop_irrelevant_columns(df):
    df = df.copy()
    dropCols = ["Utilities", "BsmtFinSF2", "PoolArea", "MoSold", "YrSold", "PoolQC"]
    df = df.drop(dropCols, axis=1)
    return df

In [20]:
def get_cleaned_data(df):
    df = fill_nans(df)
    df = drop_irrelevant_columns(df)
    return df

In [21]:
train = pd.read_csv(in_file_paths['train'], index_col='Id')
test = pd.read_csv(in_file_paths['test'], index_col='Id')

train = get_cleaned_data(train)
test = get_cleaned_data(test)

train.shape, test.shape

((1460, 74), (1459, 73))

In [22]:
train.to_csv(out_file_paths['train'])
test.to_csv(out_file_paths['test'])